## IMPORTING

In [ ]:
import geopandas as gpd
import fiona
import folium
import geopy

path = r'/Users/berkaytok/DataspellProjects/AdvGIS/data/Shelby.gdb.zip'

df = gpd.read_file(path, driver='FileGDB', layer=0)
fiona.listlayers(path)

In [ ]:
county = gpd.read_file(path, driver='FileGDB', layer='County_shelby')
schools = gpd.read_file(path, driver='FileGDB', layer='School_shelby')
libraries = gpd.read_file(path, driver='FileGDB', layer='Library_shelby')
parks = gpd.read_file(path, driver='FileGDB', layer='Parks_shelby')
fire = gpd.read_file(path, driver='FileGDB', layer='Fire_shelby')
law = gpd.read_file(path, driver='FileGDB', layer='Law_Enforcement_shelby')
libraries = gpd.read_file(path, driver='FileGDB', layer='Library_shelby')

#### SUBSETTING ROWS

In [ ]:
parks.head(15)
parks[parks["NAME"] == "May Soccer Complex"]

In [ ]:
# distance(): returns Series with minimum distance from each entry to other
# check distances of schools to parks
#schools.distance(parks).head()
# check distances of law to parks
law.distance(parks).head()

In [ ]:
# Returns a GeoSeries of points for each geometric centroid
parks.centroid.plot()

In [ ]:
# Returns a GeoSeries of (cheaply computed) points that are guaranteed to be within each geometry
schools.representative_point()

In [ ]:
# check coordinate system
schools.crs
county.crs

In [ ]:
# reproject to WGS84
schoolswgs = schools.to_crs(3857)


In [ ]:
# plot
schools.plot()


In [ ]:
# almost equals
schoolswgs.geom_almost_equals(schools)

## INTERACTIVE MAPPING

In [ ]:
county.explore()

## MAP OF SHELBY COUNTY PARKS COLORED BY PARKING AREA PER AREA TOTAL

In [ ]:
# Shelby County Parks by parking space per area and police stations in Shelby County, TN
m = parks.explore(
     column=parks["PARKING_AREA"] / parks["Shape_Area"],  # make choropleth based on "BoroName" column
     scheme="quantiles",  # use mapclassify's natural breaks scheme
     legend=True, # show legend
     k=10, # use 10 bins
     legend_kwds=dict(colorbar=False), # do not use colorbar
     name="parks" # name of the layer in the map
)
law.explore(
     m=m, # pass the map object
     color="red", # use red color on all points
     marker_kwds=dict(radius=10, fill=True), # make marker radius 10px with fill
     tooltip="FACILITY_NAME", # show "name" column in the tooltip
     tooltip_kwds=dict(labels=False), # do not show column label in the tooltip
     name="law enforcement" # name of the layer in the map
)
folium.TileLayer('Stamen Toner', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m    

## GEOMETRIC MANIPULATIONS
#### OVERLAY/INTERSECTION

In [ ]:
ax = parks.overlay(county, how='intersection').plot()
parks.plot(ax=ax, color='red', zorder=2)
county.plot(ax=ax, zorder=1)

#### DISSOLVE

In [ ]:
# dissolving the parks by the pavement type
parks.dissolve(by='PARKING_SURFMAT').plot()

## SPATIAL JOINS

In [ ]:
# law enforcement inside Shelby County parks;
parklaw = law.sjoin(parks, how="inner", predicate='intersects')
parkgrp = parklaw.groupby(['NAME']).count()
parkgrp

## GEOCODING

In [ ]:
# using the geocoding tool
shelby = parks.NAME.head()
geocoded = gpd.tools.geocode(shelby)
shelby


In [ ]:
# results are incorrect due to insufficient location names but geocoder works
geocoded

In [ ]:
shelbycounty = gpd.read_file('/Users/berkaytok/DataspellProjects/AdvGIS/data/tl_2019_47157_faces/tl_2019_47157_faces.shp')

In [65]:
shelbycounty.groupby(['TRACTCE10'])

,TFID,ATOTAL
TRACTCE10,,
000100,47357228904,6735739
000200,8211954595,1102094
000300,10769701249,2881280
000400,18225501599,2367982
000600,15773232126,5092785
...,...,...
022700,18559957990,8300707
980100,38218023724,19126877
980200,18051502312,45515855
